In [ ]:
# Import required libraries
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env file
load_dotenv()

def setup_spotify_client():
    """
    Set up and return an authenticated Spotify client
    """
    client_id = os.getenv('SPOTIFY_CLIENT_ID')
    client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
    redirect_uri = os.getenv('SPOTIFY_REDIRECT_URI')
    
    scope = [
        'user-library-read',
        'user-top-read',
        'playlist-read-private',
        'user-read-recently-played'
    ]
    
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=scope
    ))
    
    return sp

# Initialize Spotify client
sp = setup_spotify_client()

In [ ]:
# Define helper functions
def get_top_tracks(sp, time_range='medium_term', limit=50):
    """
    Get user's top tracks
    time_range: short_term (4 weeks), medium_term (6 months), long_term (all time)
    """
    results = sp.current_user_top_tracks(limit=limit, offset=0, time_range=time_range)
    
    tracks_data = []
    for track in results['items']:
        track_info = {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'popularity': track['popularity'],
            'duration_ms': track['duration_ms'],
            'preview_url': track['preview_url']
        }
        tracks_data.append(track_info)
    
    return pd.DataFrame(tracks_data)


In [ ]:
load_dotenv()

# Print to check if credentials are loading
#print("Client ID:", os.getenv('SPOTIFY_CLIENT_ID'))
#print("Client Secret:", os.getenv('SPOTIFY_CLIENT_SECRET'))
#print("Redirect URI:", os.getenv('SPOTIFY_REDIRECT_URI'))

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv('SPOTIFY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
    redirect_uri=os.getenv('SPOTIFY_REDIRECT_URI'),
    scope=['user-library-read', 'user-top-read']
))

# Test the connection
current_user = sp.current_user()
print("Connected as:", current_user['display_name'])

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Clear any existing cache
import os
if os.path.exists(".cache"):
    os.remove(".cache")

# Load environment variables
load_dotenv()

# Print environment variables (without revealing full secrets)
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIFY_REDIRECT_URI')

print(f"Client ID loaded: {'✓' if client_id else '✗'}")
print(f"Client Secret loaded: {'✓' if client_secret else '✗'}")
print(f"Redirect URI: {redirect_uri}")

# Setup Spotify client with explicit port
try:
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=['user-library-read', 'user-top-read', 'playlist-read-private', 'user-read-recently-played'],
        open_browser=True
    ))
    
    # Test the connection
    current_user = sp.current_user()
    print("\nSuccess! Connected as:", current_user['display_name'])
except Exception as e:
    print("\nError occurred:", str(e))
    print("\nTroubleshooting tips:")
    print("1. Verify your Spotify Developer Dashboard settings")
    print("2. Check if port 8080 is available")
    print("3. Ensure your credentials in .env are correct")

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import urllib3

# Disable SSL warnings and verification (for testing only)
urllib3.disable_warnings()
os.environ['PYTHONWARNINGS'] = 'ignore:Unverified HTTPS request'
os.environ['REQUESTS_CA_BUNDLE'] = ''

# Load environment variables
load_dotenv()

# Setup Spotify client
auth_manager = SpotifyOAuth(
    client_id=os.getenv('SPOTIFY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
    redirect_uri=os.getenv('SPOTIFY_REDIRECT_URI'),
    scope=['user-library-read', 'user-top-read', 'playlist-read-private', 'user-read-recently-played'],
    open_browser=False,
    requests_session=urllib3.PoolManager(cert_reqs='CERT_NONE').request
)

try:
    # Get the authorization URL
    auth_url = auth_manager.get_authorize_url()
    print("\nStep 1: Please visit this URL in your browser to authorize the application:")
    print(auth_url)
    print("\nStep 2: After authorizing, copy the URL you were redirected to")
    
    response = input("\nPaste the redirect URL here: ")
    
    # Get the token
    code = auth_manager.parse_response_code(response)
    token_info = auth_manager.get_access_token(code, check_cache=False)
    
    # Create the Spotify client
    sp = spotipy.Spotify(auth_manager=auth_manager)
    
    # Test the connection
    current_user = sp.current_user()
    print("\nSuccess! Connected as:", current_user['display_name'])

except Exception as e:
    print("\nDetailed error information:")
    print(f"Error type: {type(e).__name__}")
    print(f"Error message: {str(e)}")

In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import urllib3
import warnings

# Suppress warnings (necessary for corporate environments)
warnings.filterwarnings('ignore')
urllib3.disable_warnings()

# Load environment variables
load_dotenv()

# Setup with corporate network considerations
os.environ['REQUESTS_CA_BUNDLE'] = ''  # Ignore default certificate bundle
os.environ['PYTHONHTTPSVERIFY'] = '0'  # Disable verification for Python requests

# Initialize Spotify OAuth with modified settings
auth_manager = SpotifyOAuth(
    client_id=os.getenv('SPOTIFY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
    redirect_uri='http://127.0.0.1:9090/callback',  # Use local IP instead of localhost
    scope=['user-library-read', 'user-top-read', 'playlist-read-private', 'user-read-recently-played'],
    open_browser=False,  # Don't auto-open browser
    proxies=None  # Ignore system proxies
)

try:
    # Get auth URL
    auth_url = auth_manager.get_authorize_url()
    print("\n=== Spotify Authorization Steps ===")
    print("\n1. Copy this URL and paste it into your browser:")
    print(auth_url)
    print("\n2. Log in to Spotify if needed and authorize the application")
    print("3. After authorizing, you'll be redirected to a URL that starts with 'http://127.0.0.1:9090/callback'")
    print("4. Copy the ENTIRE redirected URL (even if the page shows an error)")
    
    response = input("\nPaste the complete redirect URL here: ")
    
    # Complete authentication
    code = auth_manager.parse_response_code(response)
    token_info = auth_manager.get_access_token(code, check_cache=False)
    
    # Initialize Spotify client
    sp = spotipy.Spotify(auth_manager=auth_manager)
    
    # Verify connection
    current_user = sp.current_user()
    print(f"\nSuccess! Connected to Spotify as: {current_user['display_name']}")
    
    # Test by getting some data
    print("\nTesting connection by fetching your top tracks...")
    top_tracks = sp.current_user_top_tracks(limit=3, time_range='short_term')
    print("\nYour top 3 tracks:")
    for idx, track in enumerate(top_tracks['items'], 1):
        print(f"{idx}. {track['name']} by {track['artists'][0]['name']}")

except Exception as e:
    print("\nError Details:")
    print(f"Type: {type(e).__name__}")
    print(f"Message: {str(e)}")
    print("\nTroubleshooting Tips for Corporate Networks:")
    print("1. Check if you can access https://accounts.spotify.com in your browser")
    print("2. Verify if you need to use your corporate proxy")
    print("3. Make sure your Spotify Developer Dashboard redirect URI matches exactly: http://127.0.0.1:9090/callback")